# Water Quality 
>- User needs to save the log files as csv 
>- Create an altered survey area to clip out the turns and wider than the lawnmower tracks. This feature will be used in clip function below.
>- Flood = North
>- Ebb = South
>- Naming Convention = EOO_Ebb_YYYYMMDD
>- User Sets csv Directory

## Import Python Modules
>- arcpy
>- os
>- shutil

In [2]:
import arcpy
import os
import shutil
from os import path

## Declare Constants
>- Working map and project through aprx
>- arcpy.env
>- Directory Paths
>- User Inputs; Month and Year

In [12]:
#ArcGIS
aprx = arcpy.mp.ArcGISProject("CURRENT")
aprx_map = aprx.listMaps("September 2021 SEJPA")[0]
ArcGIS_path = r"D:\Water_Quality\SEJPA_Ted\ArcGIS Pro"


#Environment
arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("GeoStats")
arcpy.env.workspace = r"D:\TedSmith\Water_Quality\SEJPA_Ted\ArcGIS Pro\SEJPA.gdb\March2022"
scratchWorkspace=r"D:\TedSmith\Water_Quality\SEJPA_Ted\ArcGIS Pro\SEJPA.gdb"

#User Inputs Month and Year
year_range = range(2020, 2030, 1)
month_list = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December"
]

month = input("Enter Month: ")
month = month.capitalize()
while month not in month_list:
    month = input("Please Enter a Valid Month: ")
    month = month.capitalize()

year = input("Enter Year: ")
year_int = int(year)
while year_int not in year_range:
    year = input("Please Enter a Valid Year: ")
    year_int = int(year)

#File Directories
csv_directory_path = r"D:\Water_Quality\SEJPA_Ted\CSV\{}\{}".format(year, month)
csv_directory_list = os.listdir(csv_directory_path)
rootVoxel_directory_path =r"D:\Water_Quality\SEJPA_Ted\raw_voxels"
#Coordinate System
wgs84 = 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 11258999068426.2;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
utm11 = 'PROJCS["WGS_1984_UTM_Zone_11N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-117.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-5120900 -9998100 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision'

Enter Month: September
Enter Year: 2021


## Verify Geodatabase Has Been Created

In [4]:
arcpy.env.workspace = ArcGIS_path
gdb_list = arcpy.ListWorkspaces("*", "FileGDB")
gdb_name = "{}{}".format(month,year)
gdb_path = os.path.join(ArcGIS_path, gdb_name) + '.gdb'
if gdb_path in gdb_list:
    print(f"'{gdb_name}.gdb' Has already been created\n")
else:
    arcpy.CreateFileGDB_management(ArcGIS_path, gdb_name)
    print(f"-Creating '{gdb_name}.gdb'-\n")
    
default_gdb_path = "{}\{}{}.gdb".format(ArcGIS_path, month, year)
aprx.defaultGeodatabase = default_gdb_path

'September2021.gdb' Has already been created



## Add CSV Tables to the Default GDB
>- This function adds only the needed fields

In [5]:
print(f"Adding CSVs to GDB")
arcpy.env.workspace = csv_directory_path
for csv in csv_directory_list:
    ext = csv.split(".")[1]
    if ext != 'ini':
        csv_path = os.path.join(csv_directory_path, csv)
        split = csv.split(".")[0]
        out_name = "table_{}".format(split)
        print(f"--{csv}")
        arcpy.conversion.TableToTable(csv, default_gdb_path, out_name, '', r'Latitude "Latitude" true true false 8 Double 0 0,First,#,{},Latitude,-1,-1;Longitude "Longitude" true true false 8 Double 0 0,First,#,{},Longitude,-1,-1;Time "Time" true true false 8000 Text 0 0,First,#,{},Time,0,8000;Date "Date" true true false 8 Date 0 0,{},Date,-1,-1;Heading "Magnetic Heading" true true false 255 Double 0 1,First,#,{},C Magnetic Heading,-1,-1;Heading_Check "Heading Check" true true false 255 Short 0 0,First,#,{},Batt Volts,-1,-1; Satellites "Number of Sattelites" true true false 255 Short 0 0,First,#,{},Number of Sats,-1,-1;DFS "Depth From Surface (m)" true true false 255 Float 0 2,First,#,{},DFS Depth(m),-1,-1;Temperature "Temperature (C)" true true false 255 Double 0 2,First,#,{},Temp C,-1,-1;CHL "Chlorophyll (RFU)" true true false 255 Float 0 2,First,#,{},Chl RFU,-1,-1;Conductivity "Conductivity (mS/cm)" true true false 255 Double 0 3,First,#,{},Cond mS/cm,-1,-1;Salinity "Salinity (PPT)" true true false 255 Float 0 2,First,#,{},Sal ppt,-1,-1;TDS "Total Dissolved Solids (mg/L)" true true false 255 Double 0 2,First,#,{},TDS mg/L,-1,-1;ODO "Optical Dissolved Oxygen (%)" true true false 255 Double 0 2,First,#,{},ODOsat %,-1,-1;fDom_qsu "Fluorescent Dissolved Organic Matter (QSU)" true true false 255 Float 0 2,First,#,{},fDOM (QSU),-1,-1;fDom_rfu "Fluorescent Dissolved Organic Matter (RFU)" true true false 255 Float 0 2,First,#,{},fDOM (RFU),-1,-1; BGA "Blue-Green Algae (RFU)" true true false 255 Float 0 2,First,#,{},BGA-PE RFU,-1,-1'.format(csv_path, csv_path ,csv_path,csv_path, csv_path, csv_path, csv_path ,csv_path ,csv_path, csv_path, csv_path, csv_path, csv_path, csv_path, csv_path, csv_path, csv_path),'')
print("Done")        

Adding CSVs to GDB
--EOO_Ebb_20210921.csv
--EOO_Flood_20210921.csv
--SEOO_Ebb_20210923.csv
--SEOO_Flood_20210924.csv
Done


## Create Points from Added Tables

In [6]:
print(f"Creating Points")
arcpy.env.workspace = default_gdb_path
tables_list = arcpy.ListTables()
for table in tables_list:
    point_name = table.split("_", 1)[1]
    print(f"--{table}")
    #arcpy.defense.CoordinateTableToPoint(table, point_name, 'Longitude', "DD_2", "Latitude", wgs84)
    arcpy.management.ConvertCoordinateNotation(table, point_name, "Longitude", "Latitude", "DD_2", "DD_2", None, utm11, wgs84, "INCLUDE_INVALID")
print("Done")    

Creating Points
--table_EOO_Ebb_20210921
--table_EOO_Flood_20210921
--table_SEOO_Ebb_20210923
--table_SEOO_Flood_20210924
Done


## Calculate Depth From Surface Field
>- Calculates it to have negative values

In [8]:
print(f"Calculating 'Depth From Surface' Field")
arcpy.env.workspace = default_gdb_path
points_list = arcpy.ListFeatureClasses("SEO*", "Point")
#print(points_list)
for point in points_list:
    print(f"--{point}")
    arcpy.management.CalculateField(point, 'DFS', "!DFS! * (-1)", "PYTHON3", "", "FLOAT", "NO_ENFORCE_DOMAINS")
print("Done")

Calculating 'Depth From Surface' Field
--SEOO_Ebb_20210923
--SEOO_Flood_20210924
Done


## Update Cursor
>- User inputs an approximate, to the nearest whole number, of the Magnetic Heading. 
>- Goes through the Tracks and Removes Rows that have more than 0 Satelitte Conections and are not on the heading of the undulating tracks.
>- previous tracks with stringers, the undulating track has the same heading, I am guessing this will change when we ditch the stringers. 
For this case I have added an option for two headings. My hope is that the unnecessary points will be removed.

In [10]:
arcpy.env.workspace = default_gdb_path
points_list = arcpy.ListFeatureClasses("SEOO*", "Point")
fields = ["Satellites", "Heading", "Heading_Check"]
for point in points_list:
    print(f"--{point}")
    heading_count = int(input("How many headings? 1 or 2: "))
    if heading_count == 1:
        heading_1_in = float(input("Approximate Heading? "))
        h1_hi = float(heading_1_in + 10)
        h1_lo = float(heading_1_in - 10)
        with arcpy.da.UpdateCursor(point, fields) as cursor:
            for row in cursor:
                if row[0] > 0:
                    cursor.deleteRow()

                if h1_lo < row[1] < h1_hi:
                    row[2] = 1

                if row[2] != 1:
                    cursor.deleteRow()                  


    else:
        heading_1_in = float(input("Approximate Heading of 1st Direction? "))
        h1_hi = float(heading_1_in + 10)
        h1_lo = float(heading_1_in - 10)
        heading_2_in = float(input("Approximate Heading of 2nd Direction? "))
        h2_hi = float(heading_2_in + 10)
        h2_lo = float(heading_2_in - 10)
        with arcpy.da.UpdateCursor(fc_path, fields) as cursor:
            for row in cursor:
                if row[0] > 0:
                    cursor.deleteRow()

                if h1_lo < row[1] < h1_hi:
                    row[2] = 1

                if h2_lo < row[1] < h2_hi:
                    row[2] = 1

                if row[2] == 1:
                    cursor.deleteRow()

print("Done")                

--SEOO_Ebb_20210923
How many headings? 1 or 2: 1
Approximate Heading? 155
--SEOO_Flood_20210924
How many headings? 1 or 2: 1
Approximate Heading? 335
Done


## Verify Voxel Directories

In [13]:
arcpy.env.workspace = default_gdb_path
points_list = arcpy.ListFeatureClasses("EOO*", "Point")
for point in points_list:
    split = point.split("_")
    name = split[0] +"_"+ split[1]
    year_directory_path = os.path.join(rootVoxel_directory_path, year)
    month_directory_path = os.path.join(year_directory_path, month)
    studyArea_directory_path = os.path.join(month_directory_path, name)
    yde = os.path.exists(year_directory_path)
    mde = os.path.exists(month_directory_path)
    sde = os.path.exists(studyArea_directory_path)

    try:
        if yde == False:
            print(f"--Creating {year} Directory")
            os.mkdir(year_directory_path)
    except Exception as e:
        print("Year Directory Already Created")
        
    try:
        if mde == False:
            print(f"--Creating {month} Directory")
            os.mkdir(month_directory_path)
    except Exception as e:
        print("Month Directory Already Created")
            
    try:    
        if sde == False:
            print(f"--Creating {name} Directory")
            os.mkdir(studyArea_directory_path)
    except Exception as e:
        print("Study Area Directory Already Created")


--Creating EOO_Ebb Directory
--Creating EOO_Flood Directory


## Create Minimum Bounding Geometry
This polygon will act as a mask for the voxel

In [14]:
arcpy.env.workspace = default_gdb_path
points_list = arcpy.ListFeatureClasses("EO*", "Point")
print(f"Creating Minimum Bounding Geometry")
for point in points_list:
    print(f"--{point}")
    split = point.split("_")
    out_name = split[0]+"_"+split[1]+"_Boundary"
    arcpy.management.MinimumBoundingGeometry(point, out_name, "CONVEX_HULL")
print("Done")

Creating Minimum Bounding Geometry
--EOO_Ebb_20210921
--EOO_Flood_20210921
Done


## 3D EBK and Voxel Production
This chunk has 9 functions, one for each variable requested  

In [16]:
def TempC(point_path):  
    # Model
    ebk_name = point + "_TempC_EBK3D"
    print(f"Creating Temperature 3D EBK for {point}")
    arcpy.ga.EmpiricalBayesianKriging3D(in_features= point_path,
                                        elevation_field= "DFS",
                                        value_field="Temperature",
                                        out_ga_layer= ebk_name,
                                        elevation_units= "METER",
                                        measurement_error_field= "",
                                        semivariogram_model_type= "EXPONENTIAL",
                                        transformation_type= "EMPIRICAL",
                                        subset_size=500,
                                        overlap_factor=5,
                                        number_simulations=100,
                                        trend_removal="FIRST",
                                        elev_inflation_factor= "",
                                        search_neighborhood="NBRTYPE=Standard3D RADIUS=nan NBR_MAX=2 NBR_MIN=1 SECTOR_TYPE=TWELVE_SECTORS",
                                        output_elevation=None,
                                        output_type="PREDICTION",
                                        quantile_value="",
                                        threshold_type="NOT_EXCEED",
                                        probability_threshold="")
        
    #Voxels
    voxel_name = point + "_TempC.nc"
    print(f"Creating Temperature voxel for {point}")
    arcpy.ga.GALayer3DToNetCDF(in_3d_geostat_layers=ebk_name,
                               out_netcdf_file=os.path.join(voxel_directory_path, voxel_name),
                               export_locations= "3D_GRIDDED_POINTS", 
                               x_spacing= "10 Meters", 
                               y_spacing= "7 Meters", 
                               elevation_spacing= "0.325 Meters", 
                               in_study_area= polygon)
    print(f"{point}'s {value_field} voxel has been placed in {voxel_directory_path}\n")

    
    
def Cond(point_path):  # Model
    ebk_name = point + "_Cond_EBK3D"
    print(f"Creating Conductivity 3D EBK for {point}")
    arcpy.ga.EmpiricalBayesianKriging3D(in_features= point_path,
                                        elevation_field= "DFS",
                                        value_field="Conductivity",
                                        out_ga_layer= ebk_name,
                                        elevation_units= "METER",
                                        measurement_error_field= "",
                                        semivariogram_model_type= "EXPONENTIAL",
                                        transformation_type= "LOGEMPIRICAL",
                                        subset_size=500,
                                        overlap_factor=5,
                                        number_simulations=100,
                                        trend_removal="FIRST",
                                        elev_inflation_factor="",
                                        search_neighborhood="NBRTYPE=Standard3D RADIUS=nan NBR_MAX=2 NBR_MIN=1 SECTOR_TYPE=TWELVE_SECTORS",
                                        output_elevation=None,
                                        output_type="PREDICTION",
                                        quantile_value="",
                                        threshold_type="NOT_EXCEED",
                                        probability_threshold="")
        
    #Voxels
    voxel_name = point + "_Cond.nc"
    print(f"Creating Conductivity voxel for {point}")
    arcpy.ga.GALayer3DToNetCDF(in_3d_geostat_layers=ebk_name,
                               out_netcdf_file=os.path.join(voxel_directory_path, voxel_name),
                               export_locations= "3D_GRIDDED_POINTS", 
                               x_spacing= "10 Meters", 
                               y_spacing= "7 Meters", 
                               elevation_spacing= ".325 Meters", 
                               in_study_area= polygon)
    print(f"{point}'s {value_field} voxel has been placed in {voxel_directory_path}\n")

    
    
    
def Sal(point_path):  # Model
    ebk_name = point + "_Sal_ppt_EBK3D"
    print(f"Creating Salinity 3D EBK for {point}")
    arcpy.ga.EmpiricalBayesianKriging3D(in_features= point_path,
                                        elevation_field= "DFS",
                                        value_field="Salinity",
                                        out_ga_layer= ebk_name,
                                        elevation_units= "METER",
                                        measurement_error_field= "",
                                        semivariogram_model_type= "EXPONENTIAL",
                                        transformation_type= "LOGEMPIRICAL",
                                        subset_size=500,
                                        overlap_factor=5,
                                        number_simulations=100,
                                        trend_removal="FIRST",
                                        elev_inflation_factor="",
                                        search_neighborhood="NBRTYPE=Standard3D RADIUS=nan NBR_MAX=2 NBR_MIN=1 SECTOR_TYPE=TWELVE_SECTORS",
                                        output_elevation=None,
                                        output_type="PREDICTION",
                                        quantile_value=0.5,
                                        threshold_type="EXCEED",
                                        probability_threshold=None)
        
    #Voxels
    voxel_name = point + "_Sal.nc"
    print(f"Creating Salinity voxel for {point}")
    arcpy.ga.GALayer3DToNetCDF(in_3d_geostat_layers=ebk_name,
                               out_netcdf_file=os.path.join(voxel_directory_path, voxel_name),
                               export_locations= "3D_GRIDDED_POINTS", 
                               x_spacing= "10 Meters", 
                               y_spacing= "7 Meters", 
                               elevation_spacing= "0.325 Meters", 
                               in_study_area= polygon)
    print(f"{point}'s {value_field} voxel has been placed in {voxel_directory_path}\n")



def TDS(point_path):  # Model
    ebk_name = point + "_TDS_EBK3D"
    print(f"Creating TDS 3D EBK for {point}")
    arcpy.ga.EmpiricalBayesianKriging3D(in_features= point_path,
                                        elevation_field= "DFS",
                                        value_field="TDS",
                                        out_ga_layer= ebk_name,
                                        elevation_units= "METER",
                                        measurement_error_field= "",
                                        semivariogram_model_type= "EXPONENTIAL",
                                        transformation_type= "EMPIRICAL",
                                        subset_size=500,
                                        overlap_factor=5,
                                        number_simulations=100,
                                        trend_removal="FIRST",
                                        elev_inflation_factor="",
                                        search_neighborhood="NBRTYPE=Standard3D RADIUS=nan NBR_MAX=2 NBR_MIN=1 SECTOR_TYPE=TWELVE_SECTORS",
                                        output_elevation=None,
                                        output_type="PREDICTION",
                                        quantile_value="",
                                        threshold_type="EXCEED",
                                        probability_threshold="")
        
    #Voxels
    voxel_name = point + "_TDS.nc"
    print(f"Creating TDS voxel for {point}")
    arcpy.ga.GALayer3DToNetCDF(in_3d_geostat_layers=ebk_name,
                               out_netcdf_file=os.path.join(voxel_directory_path, voxel_name),
                               export_locations= "3D_GRIDDED_POINTS", 
                               x_spacing= "10 Meters", 
                               y_spacing= "7 Meters", 
                               elevation_spacing= "0.325 Meters", 
                               in_study_area= polygon)
    print(f"{point}'s {value_field} voxel has been placed in {voxel_directory_path}\n")



def fDOM_qsu(point_path):  # Model
    ebk_name = point + "_fDOM_qsu_EBK3D"
    print(f"Creating fDOM (qsu) 3D EBK for {point}")
    arcpy.ga.EmpiricalBayesianKriging3D(in_features= point_path,
                                        elevation_field= "DFS",
                                        value_field="fDom_qsu",
                                        out_ga_layer= ebk_name,
                                        elevation_units= "METER",
                                        measurement_error_field= "",
                                        semivariogram_model_type= "EXPONENTIAL",
                                        transformation_type= "NONE",
                                        subset_size=500,
                                        overlap_factor=5,
                                        number_simulations=100,
                                        trend_removal="FIRST",
                                        elev_inflation_factor= "",
                                        search_neighborhood="NBRTYPE=Standard3D RADIUS=nan NBR_MAX=2 NBR_MIN=1 SECTOR_TYPE=TWELVE_SECTORS",
                                        output_elevation=None,
                                        output_type="PREDICTION",
                                        quantile_value="",
                                        threshold_type="NOT_EXCEED",
                                        probability_threshold="")
        
    #Voxels
    voxel_name = point + "_fDOM_qsu.nc"
    print(f"Creating fDOM (qsu) voxel for {point}")
    arcpy.ga.GALayer3DToNetCDF(in_3d_geostat_layers=ebk_name,
                               out_netcdf_file=os.path.join(voxel_directory_path, voxel_name),
                               export_locations= "3D_GRIDDED_POINTS", 
                               x_spacing= "10 Meters", 
                               y_spacing= "7 Meters", 
                               elevation_spacing= "0.325 Meters", 
                               in_study_area= polygon)
    print(f"{point}'s {value_field} voxel has been placed in {voxel_directory_path}\n")




def fDOM_rfu(point_path):  # Model
    ebk_name = point + "_fDOM_rfu_EBK3D"
    print(f"Creating fDom_rfu 3D EBK for {point}")
    arcpy.ga.EmpiricalBayesianKriging3D(in_features= point_path,
                                        elevation_field= "DFS",
                                        value_field="fDom_rfu",
                                        out_ga_layer= ebk_name,
                                        elevation_units= "METER",
                                        measurement_error_field= "",
                                        semivariogram_model_type= "EXPONENTIAL",
                                        transformation_type= "NONE",
                                        subset_size=500,
                                        overlap_factor=5,
                                        number_simulations=100,
                                        trend_removal="FIRST",
                                        elev_inflation_factor="",
                                        search_neighborhood="NBRTYPE=Standard3D RADIUS=nan NBR_MAX=2 NBR_MIN=1 SECTOR_TYPE=TWELVE_SECTORS",
                                        output_elevation=None,
                                        output_type="PREDICTION",
                                        quantile_value="",
                                        threshold_type="",
                                        probability_threshold="")
        
    #Voxels
    voxel_name = point + "_fDOM_rfu.nc"
    print(f"Creating fDOM (rfu) voxel for {point}")
    arcpy.ga.GALayer3DToNetCDF(in_3d_geostat_layers=ebk_name,
                               out_netcdf_file=os.path.join(voxel_directory_path, voxel_name),
                               export_locations= "3D_GRIDDED_POINTS", 
                               x_spacing= "10 Meters", 
                               y_spacing= "7 Meters", 
                               elevation_spacing= "0.325 Meters", 
                               in_study_area= polygon)
    print(f"{point}'s {value_field} voxel has been placed in {voxel_directory_path}\n")



def BGA(point_path):  # Model
    ebk_name = point + "_BGA_EBK3D"
    print(f"Creating Blue-Green Algae 3D EBK for {point}")
    arcpy.ga.EmpiricalBayesianKriging3D(in_features= point_path,
                                        elevation_field= "DFS",
                                        value_field="BGA",
                                        out_ga_layer= ebk_name,
                                        elevation_units= "METER",
                                        measurement_error_field= "",
                                        semivariogram_model_type= "EXPONENTIAL",
                                        transformation_type= "EMPIRICAL",
                                        subset_size=500,
                                        overlap_factor=5,
                                        number_simulations=100,
                                        trend_removal="FIRST",
                                        elev_inflation_factor="",
                                        search_neighborhood="NBRTYPE=Standard3D RADIUS=nan NBR_MAX=2 NBR_MIN=1 SECTOR_TYPE=TWELVE_SECTORS",
                                        output_elevation=None,
                                        output_type="PREDICTION",
                                        quantile_value="",
                                        threshold_type="",
                                        probability_threshold=None)
        
    #Voxels
    voxel_name = point + "_BGA.nc"
    print(f"Creating Blue-Green Algae voxel for {point}")
    arcpy.ga.GALayer3DToNetCDF(in_3d_geostat_layers=ebk_name,
                               out_netcdf_file=os.path.join(voxel_directory_path, voxel_name),
                               export_locations= "3D_GRIDDED_POINTS", 
                               x_spacing= "10 Meters", 
                               y_spacing= "7 Meters", 
                               elevation_spacing= "0.325 Meters", 
                               in_study_area= polygon)
    print(f"{point}'s {value_field} voxel has been placed in {voxel_directory_path}\n")

    
    
def ODO_perc(point_path): # Model
    ebk_name = point + "_ODO_perc_EBK3D"
    print(f"Creating Optical Dissolved Oxygen % 3D EBK for {point}")
    arcpy.ga.EmpiricalBayesianKriging3D(in_features= point_path,
                                        elevation_field= "DFS",
                                        value_field="ODO",
                                        out_ga_layer= ebk_name,
                                        elevation_units= "METER",
                                        measurement_error_field= "",
                                        semivariogram_model_type="EXPONENTIAL",
                                        transformation_type="EMPIRICAL",
                                        subset_size=1000,
                                        overlap_factor=5,
                                        number_simulations=100,
                                        trend_removal="FIRST",
                                        elev_inflation_factor="",
                                        search_neighborhood="NBRTYPE=Standard3D RADIUS=nan NBR_MAX=2 NBR_MIN=1 SECTOR_TYPE=TWELVE_SECTORS",
                                        output_elevation=None,
                                        output_type="PREDICTION",
                                        quantile_value="",
                                        threshold_type="",
                                        probability_threshold="")
        
    #Voxels
    voxel_name = point + "_ODO_perc.nc"
    print(f"Creating Optical Dissolved Oxygen % voxel for {point}")
    arcpy.ga.GALayer3DToNetCDF(in_3d_geostat_layers=ebk_name,
                               out_netcdf_file=os.path.join(voxel_directory_path, voxel_name),
                               export_locations= "3D_GRIDDED_POINTS", 
                               x_spacing= "10 Meters", 
                               y_spacing= "7 Meters", 
                               elevation_spacing= "0.325 Meters", 
                               in_study_area= polygon)
    print(f"{point}'s {value_field} voxel has been placed in {voxel_directory_path}\n")


def CHL_rfu(point_path):  # Model
    ebk_name = point + "_CHL_rfu_EBK3D"
    print(f"Creating Chlorophyll fluorescence 3D EBK for {point}")
    arcpy.ga.EmpiricalBayesianKriging3D(in_features= point_path,
                                        elevation_field= "DFS",
                                        value_field="CHL",
                                        out_ga_layer= ebk_name,
                                        elevation_units= "METER",
                                        measurement_error_field= "",
                                        semivariogram_model_type= "EXPONENTIAL",
                                        transformation_type= "EMPIRICAL",
                                        subset_size=500,
                                        overlap_factor=5,
                                        number_simulations=100,
                                        trend_removal="FIRST",
                                        elev_inflation_factor="",
                                        search_neighborhood="NBRTYPE=Standard3D RADIUS=nan NBR_MAX=2 NBR_MIN=1 SECTOR_TYPE=TWELVE_SECTORS",
                                        output_elevation=None,
                                        output_type="PREDICTION",
                                        quantile_value="",
                                        threshold_type="",
                                        probability_threshold=None)
    print(f"Chlorophyll fluorescence 3D EBK has been created for {point}")
        
    #Voxels
    voxel_name = point + "_CHL.nc"
    print(f"Creating Chlorophyll fluorescence voxel for {point}")
    arcpy.ga.GALayer3DToNetCDF(in_3d_geostat_layers=ebk_name,
                               out_netcdf_file=os.path.join(voxel_directory_path, voxel_name),
                               export_locations= "3D_GRIDDED_POINTS", 
                               x_spacing= "10 Meters", 
                               y_spacing= "7 Meters", 
                               elevation_spacing= "0.325 Meters", 
                               in_study_area= polygon)
    print(f"{point}'s {value_field} voxel has been placed in {voxel_directory_path}\n")


    
if __name__ == "__main__":
    arcpy.env.workspace = default_gdb_path
    points_list = arcpy.ListFeatureClasses("EO*", "Point")
    polygons_list = arcpy.ListFeatureClasses("EO*", "Polygon")
    for point in points_list:
        pt_split = point.split("_")
        name = pt_split[0] +"_"+ pt_split[1]
        voxel_directory_path = r"{}\{}\{}\{}".format(rootVoxel_directory_path,year, month, name)
        for polygon in polygons_list:
            pg_split = polygon.split("_")
            pg_check = pg_split[0] +"_"+ pg_split[1]
            if name == pg_check:
                point_path = os.path.join(default_gdb_path, point)
                '''TempC(point_path)
                Cond(point_path)
                Sal(point_path)
                TDS(point_path)
                fDOM_qsu(point_path)
                fDOM_rfu(point_path)
                BGA(point_path)'''
                ODO_perc(point_path)
                #CHL_rfu(point_path)

print("Done")

Creating Optical Dissolved Oxygen % 3D EBK for EOO_Ebb_20210921
Creating Optical Dissolved Oxygen % voxel for EOO_Ebb_20210921


NameError: name 'value_field' is not defined